In [5]:

import pandas as pd
import numpy as np
from numpy import mean
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
#from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from imblearn.ensemble import EasyEnsembleClassifier, RUSBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
# Load the dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
data = pd.read_csv(url, sep=";")
# Binarize the target variable
data['quality'] = [1 if x >= 7 else 0 for x in data['quality']]
# Split the data into training and test sets
x_data = data.drop('quality', axis=1)
y_data = data['quality']
pos_weight = float(y_data.value_counts()[0]) / y_data.value_counts()[1]


# append models
models = []
models.append(('CatB Base', CatBoostClassifier(verbose=0)))
models.append(('CatB weighted', CatBoostClassifier(verbose=0, class_weights = (1, 7) )))
models.append(('lgbm Base', LGBMClassifier(verbose=-1)))
models.append(('lgbm weighted', LGBMClassifier(verbose=-1, is_unbalance=True)))
models.append(('xgb Base', XGBClassifier()))
models.append(('xgb weighted', XGBClassifier(scale_pos_weight=pos_weight)))
models.append(('bagging ', BaggingClassifier(n_estimators=50, random_state=0)))
models.append(('bagging_balanced', BalancedBaggingClassifier(n_estimators=50, random_state=0)))
models.append(('RF', RandomForestClassifier()))
models.append(('RF_balanced', BalancedRandomForestClassifier(sampling_strategy="all",replacement=True,bootstrap=False,)))
models.append(('ADA',  AdaBoostClassifier()))
eec_estimator = AdaBoostClassifier(algorithm="SAMME")
models.append(('eec', EasyEnsembleClassifier(estimator=eec_estimator)))
models.append(('rusboost',  RUSBoostClassifier(estimator=eec_estimator)))
models.append(('SVM linear weighted', SVC(probability=True,kernel='linear', class_weight='balanced')))
models.append(('SVM rbf weighted', SVC(probability=True,kernel='rbf', class_weight='balanced')))
models.append(('SVM poly weighted', SVC(probability=True,kernel='poly', class_weight='balanced')))
models.append(('SVM sigmoid weighted', SVC(probability=True,kernel='sigmoid', class_weight='balanced')))
results = []
results_std = []
names = []
scoring = 'f1'
print(scoring)
# Set Cross-Validation
#Remember: Better use Stratified Kfold, if advised
kfold = model_selection.StratifiedKFold(n_splits=10, random_state=None)
for name, model in models:   
    cv_results = model_selection.cross_val_score(model, x_data, y_data, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    #results_std.append(cv_results.std())
    mean_scoring = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(mean_scoring)



f1
CatB Base: 0.395477 (0.081732)
CatB weighted: 0.507460 (0.087083)
lgbm Base: 0.440398 (0.062896)
lgbm weighted: 0.465210 (0.066694)
xgb Base: 0.461105 (0.061125)
xgb weighted: 0.472599 (0.089138)
bagging : 0.429348 (0.092314)
bagging_balanced: 0.508905 (0.097122)
RF: 0.418648 (0.109219)
RF_balanced: 0.490692 (0.091606)
ADA: 0.431315 (0.081437)
eec: 0.515129 (0.098673)
rusboost: 0.486575 (0.096657)
SVM linear weighted: 0.488557 (0.095099)
SVM rbf weighted: 0.355290 (0.067245)
SVM poly weighted: 0.284511 (0.019952)
SVM sigmoid weighted: 0.246529 (0.059145)
